In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install libs
!pip -q install torch==1.5.0
!pip -q install torchtext==0.6

In [ ]:
!pip -q install thai2transformers

     |██████████████████████████████▎ | 834.1 MB 1.5 MB/s eta 0:00:33tcmalloc: large alloc 1147494400 bytes == 0x55f8d9826000 @  0x7fdc503b1615 0x55f8d61aa4cc 0x55f8d628a47a 0x55f8d61ad2ed 0x55f8d629ee1d 0x55f8d6220e99 0x55f8d621b9ee 0x55f8d61aebda 0x55f8d6220d00 0x55f8d621b9ee 0x55f8d61aebda 0x55f8d621d737 0x55f8d629fc66 0x55f8d621cdaf 0x55f8d629fc66 0x55f8d621cdaf 0x55f8d629fc66 0x55f8d621cdaf 0x55f8d61af039 0x55f8d61f2409 0x55f8d61adc52 0x55f8d6220c25 0x55f8d621b9ee 0x55f8d61aebda 0x55f8d621d737 0x55f8d621b9ee 0x55f8d61aebda 0x55f8d621c915 0x55f8d61aeafa 0x55f8d621cc0d 0x55f8d621b9ee
     |████████████████████████████████| 881.9 MB 20 kB/s 
     |████████████████████████████████| 1.3 MB 34.6 MB/s 
     |████████████████████████████████| 1.1 MB 30.9 MB/s 
     |████████████████████████████████| 170 kB 28.2 MB/s 
     |████████████████████████████████| 11.0 MB 22.6 MB/s 
     |████████████████████████████████| 298 kB 30.7 MB/s 
     |████████████████████████████████| 8.7 MB 17.0 MB/s 

In [ ]:
fin = open("/usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py")
content = []
for line in fin:
  content.append(line)
fin.close()

content[39] = "    SAVE_STATE_WARNING = ''\n"

fout = open("/usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py", "w")
for line in content:
  fout.write(line)
fout.close()

In [ ]:
# from packaging import version
# import torch
# if version.parse(torch.__version__) <= version.parse("1.4.1"):
#     SAVE_STATE_WARNING = ""
# else:
#     from torch.optim.lr_scheduler import SAVE_STATE_WARNING

In [ ]:
# !pip install -q pytorch-lightning

In [ ]:
# !pip -q install git+https://github.com/PyTorchLightning/pytorch-lightning
# import pytorch_lightning as pl
# print(pl.__version__)

# Load Pre-trained Model

In [40]:
cd /content/drive/MyDrive/Mispelling/misspelling-semantics/

/content/drive/MyDrive/Mispelling/misspelling-semantics


In [41]:
DIR = "Models/WangchanBERTa-exp4"

In [42]:
model_idx = 0
model_name = "wangchanberta"
# model_name = "xlmr"
# model_name = "mbert"
args_params = f"{model_name} wisesight_sentiment {DIR}/Outputs/ {DIR}/Logs/ --batch_size 8 --seed {model_idx} --run_name exp1 --num_train_epochs 10"

In [43]:
import argparse
import math
import os
from functools import partial
import urllib.request
from tqdm import tqdm
from typing import Collection, Callable
from pathlib import Path
from sklearn import preprocessing
import pandas as pd
import numpy as np

import torch
from transformers import (
    AdamW, 
    get_linear_schedule_with_warmup, 
    get_constant_schedule, 
    AutoTokenizer, 
    AutoModel,
    AutoModelForSequenceClassification, 
    AutoConfig,
    Trainer, 
    TrainingArguments,
    CamembertTokenizer,
    BertTokenizer,
    BertTokenizerFast,
    BertConfig,
    XLMRobertaTokenizer,
    XLMRobertaTokenizerFast,
    XLMRobertaConfig,
    DataCollatorWithPadding,
    default_data_collator
)

from datasets import load_dataset, list_metrics, load_dataset, Dataset
from thai2transformers.datasets import SequenceClassificationDataset
from thai2transformers.metrics import classification_metrics, multilabel_classification_metrics
from thai2transformers.finetuners import SequenceClassificationFinetuner
from thai2transformers.auto import AutoModelForMultiLabelSequenceClassification
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
)
from thai2transformers.utils import get_dict_val
from thai2transformers.conf import Task
from thai2transformers import preprocess

CACHE_DIR = f'{str(Path.home())}/.cache/huggingface_datasets'

METRICS = {
    Task.MULTICLASS_CLS: classification_metrics,
    Task.MULTILABEL_CLS: multilabel_classification_metrics
}

PUBLIC_MODEL = {
    # 'mbert': {
    #     'name': 'bert-base-multilingual-cased',
    #     'tokenizer': BertTokenizerFast.from_pretrained('bert-base-multilingual-cased'),
    #     'config': BertConfig.from_pretrained('bert-base-multilingual-cased'),
    # },
    'xlmr': {
        'name': 'xlm-roberta-base',
        'tokenizer': XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base'),
        'config': XLMRobertaConfig.from_pretrained('xlm-roberta-base'),
    },
    # 'xlmr-large': {
    #     'name': 'xlm-roberta-large',
    #     'tokenizer': XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large'),
    #     'config': XLMRobertaConfig.from_pretrained('xlm-roberta-base'),
    # },
    # 'thbert': {
    #     'name': 'monsoon-nlp/bert-base-thai',
    #     'tokenizer': AutoTokenizer.from_pretrained('monsoon-nlp/bert-base-thai'),
    #     'config': AutoConfig.from_pretrained('xlm-roberta-base'),
    # },
}

TOKENIZER_CLS = {
    'wangchanberta': CamembertTokenizer,
    # 'spm': ThaiRobertaTokenizer,
    # 'newmm': ThaiWordsNewmmTokenizer,
    # 'syllable': ThaiWordsSyllableTokenizer,
    # 'sefr_cut': FakeSefrCutTokenizer,
}

DATASET_METADATA = {
    'wisesight_sentiment': {
        'huggingface_dataset_name': 'wisesight_sentiment',
        'task': Task.MULTICLASS_CLS,
        'text_input_col_name': 'texts',
        'label_col_name': 'category',
        'num_labels': 3,
        'split_names': ['train', 'validation', 'test']
    }
}

def init_public_model_tokenizer_for_seq_cls(public_model_name, task, num_labels):
    
    config = PUBLIC_MODEL[public_model_name]['config']
    config.num_labels = num_labels
    tokenizer = PUBLIC_MODEL[public_model_name]['tokenizer']
    model_name = PUBLIC_MODEL[public_model_name]['name']
    if task == Task.MULTICLASS_CLS:
        model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                   config=config)
    if task == Task.MULTILABEL_CLS:
        model = AutoModelForMultiLabelSequenceClassification.from_pretrained(model_name,
                                                                             config=config)

    # print(f'\n[INFO] Model architecture: {model} \n\n')
    # print(f'\n[INFO] tokenizer: {tokenizer} \n\n')

    return model, tokenizer, config

def init_model_tokenizer_for_seq_cls(model_dir, tokenizer_cls, tokenizer_dir, task, num_labels):
    
    config = AutoConfig.from_pretrained(
        model_dir,
        num_labels=num_labels
    );

    tokenizer = tokenizer_cls.from_pretrained(
        tokenizer_dir,
    );

    if task == Task.MULTICLASS_CLS:
        model = AutoModelForSequenceClassification.from_pretrained(
            model_dir,
            config=config,
        )
    elif task == Task.MULTILABEL_CLS:
        model = AutoModelForMultiLabelSequenceClassification.from_pretrained(
            model_dir,
            config=config,
        )

    # print(f'\n[INFO] Model architecture: {model} \n\n')
    # print(f'\n[INFO] tokenizer: {tokenizer} \n\n')

    return model, tokenizer, config

def init_trainer(task, model, train_dataset, val_dataset, warmup_steps, args, data_collator=default_data_collator): 
        
    training_args = TrainingArguments(
                        num_train_epochs=args.num_train_epochs,
                        per_device_train_batch_size=args.batch_size,
                        per_device_eval_batch_size=args.batch_size,
                        gradient_accumulation_steps=args.gradient_accumulation_steps,
                        learning_rate=args.learning_rate,
                        warmup_steps=warmup_steps,
                        weight_decay=args.weight_decay,
                        adam_epsilon=args.adam_epsilon,
                        max_grad_norm=args.max_grad_norm,
                        #checkpoint
                        output_dir=args.output_dir,
                        overwrite_output_dir=True,
                        #logs
                        logging_dir=args.log_dir,
                        logging_first_step=False,
                        logging_steps=args.logging_steps,
                        #eval
                        evaluation_strategy='epoch' if 'validation' in DATASET_METADATA[args.dataset_name]['split_names'] else 'no',
                        load_best_model_at_end=True,
                        #others
                        seed=args.seed,
                        fp16=args.fp16,
                        fp16_opt_level=args.fp16_opt_level,
                        dataloader_drop_last=False,
                        no_cuda=args.no_cuda,
                        metric_for_best_model=args.metric_for_best_model,
                        prediction_loss_only=False,
                        run_name=args.run_name
                    )
    if task == Task.MULTICLASS_CLS:
        compute_metrics_fn = METRICS[task]
    elif task == Task.MULTILABEL_CLS:
        compute_metrics_fn = partial(METRICS[task],n_labels=DATASET_METADATA[args.dataset_name]['num_labels'])

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics_fn,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator
    )
    return trainer, training_args

# def _process_transformers(
#     text: str,
#     pre_rules: Collection[Callable] = [
#         preprocess.fix_html,
#         preprocess.rm_brackets,
#         preprocess.replace_newlines,
#         preprocess.rm_useless_spaces,
#         preprocess.replace_spaces,
#         preprocess.replace_rep_after,
#     ],
#     tok_func: Callable = preprocess.word_tokenize,
#     post_rules: Collection[Callable] = [preprocess.ungroup_emoji, preprocess.replace_wrep_post],
#     lowercase: bool = False
# ) -> str:
#     if lowercase:
#         text = text.lower()
#     for rule in pre_rules:
#         text = rule(text)
#     toks = tok_func(text)
#     for rule in post_rules:
#         toks = rule(toks)
#     return "".join(toks)

In [44]:
parser = argparse.ArgumentParser()
# Required
parser.add_argument('tokenizer_type_or_public_model_name', type=str, help='The type token model used. Specify the name of tokenizer either `spm`, `newmm`, `syllable`, or `sefr_cut`.')
parser.add_argument('dataset_name', help='Specify the dataset name to finetune. Currently, sequence classification datasets include `wisesight_sentiment`, `generated_reviews_enth-correct_translation`, `generated_reviews_enth-review_star` and`wongnai_reviews`.')
parser.add_argument('output_dir', type=str)
parser.add_argument('log_dir', type=str)

parser.add_argument('--model_dir', type=str)
parser.add_argument('--tokenizer_dir', type=str)
parser.add_argument('--prepare_for_tokenization', action='store_true', default=False, help='To replace space with a special token e.g. `<_>`. This may require for some pretrained models.')
parser.add_argument('--space_token', type=str, default=' ', help='The special token for space, specify if argumet: prepare_for_tokenization is applied')
parser.add_argument('--max_length', type=int, default=None)
parser.add_argument('--lowercase', action='store_true', default=False)

# Finetuning
parser.add_argument('--num_train_epochs', type=int, default=5)
parser.add_argument('--learning_rate', type=float, default=1e-05)
parser.add_argument('--weight_decay', type=float, default=0.01)
parser.add_argument('--warmup_ratio', type=float, default=0.1)
parser.add_argument('--batch_size', type=int, default=16)
parser.add_argument('--no_cuda', action='store_true', default=False)
parser.add_argument('--fp16', action='store_true', default=False)
parser.add_argument('--greater_is_better', action='store_true', default=True)
parser.add_argument('--metric_for_best_model', type=str, default='f1_micro')
parser.add_argument('--logging_steps', type=int, default=10)
parser.add_argument('--seed', type=int, default=2020)
parser.add_argument('--fp16_opt_level', type=str, default='O1')
parser.add_argument('--gradient_accumulation_steps', type=int, default=1)
parser.add_argument('--adam_epsilon', type=float, default=1e-08)
parser.add_argument('--max_grad_norm', type=float, default=1.0)

# wandb
parser.add_argument('--run_name', type=str, default=None)

_StoreAction(option_strings=['--run_name'], dest='run_name', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [45]:
# parser.add_argument('tokenizer_type_or_public_model_name', type=str, help='The type token model used. Specify the name of tokenizer either `spm`, `newmm`, `syllable`, or `sefr_cut`.')
# parser.add_argument('dataset_name', help='Specify the dataset name to finetune. Currently, sequence classification datasets include `wisesight_sentiment`, `generated_reviews_enth-correct_translation`, `generated_reviews_enth-review_star` and`wongnai_reviews`.')
# parser.add_argument('output_dir', type=str)
# parser.add_argument('log_dir', type=str)
args = parser.parse_args(args_params.split())

# Set seed
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(args.seed)
np.random.seed(args.seed)

In [46]:

# try:
print(f'\n\n[INFO] Dataset: {args.dataset_name}')
print(f'\n\n[INFO] Huggingface\'s dataset name: {DATASET_METADATA[args.dataset_name]["huggingface_dataset_name"]} ')
print(f'[INFO] Task: {DATASET_METADATA[args.dataset_name]["task"].value}')
print(f'\n[INFO] space_token: {args.space_token}')
print(f'[INFO] prepare_for_tokenization: {args.prepare_for_tokenization}\n')




[INFO] Dataset: wisesight_sentiment


[INFO] Huggingface's dataset name: wisesight_sentiment 
[INFO] Task: multiclass_classification

[INFO] space_token:  
[INFO] prepare_for_tokenization: False



In [47]:
# dataset = load_dataset(DATASET_METADATA[args.dataset_name]["huggingface_dataset_name"])

In [48]:
# labels = {
#     "neg": 2,
#     "neu": 1,
#     "pos": 0,
#     "q": 3
# }
# dataset

In [49]:
# dataset["train"][0:5]

In [50]:
text_input_col_name = DATASET_METADATA[args.dataset_name]['text_input_col_name']

if args.tokenizer_type_or_public_model_name not in list(TOKENIZER_CLS.keys()) \
    and args.tokenizer_type_or_public_model_name not in list(PUBLIC_MODEL.keys()):
    raise f"The tokenizer type or public model name `{args.tokenizer_type_or_public_model_name}`` is not supported"

if args.tokenizer_type_or_public_model_name in list(TOKENIZER_CLS.keys()):
    tokenizer_cls = TOKENIZER_CLS[args.tokenizer_type_or_public_model_name]


task = DATASET_METADATA[args.dataset_name]['task']
if args.tokenizer_type_or_public_model_name in PUBLIC_MODEL.keys():
    print(args.tokenizer_type_or_public_model_name)
    model, tokenizer, config = init_public_model_tokenizer_for_seq_cls(args.tokenizer_type_or_public_model_name,
                                                        task=task,
                                                        num_labels=DATASET_METADATA[args.dataset_name]['num_labels']);
else:
    print("WangchanBERTa")
    model, tokenizer, config = init_model_tokenizer_for_seq_cls("airesearch/wangchanberta-base-att-spm-uncased",
                                                        tokenizer_cls,
                                                        "airesearch/wangchanberta-base-att-spm-uncased",
                                                        task=task,
                                                        num_labels=DATASET_METADATA[args.dataset_name]['num_labels']);

WangchanBERTa


Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [51]:
# if args.tokenizer_type_or_public_model_name == 'wangchanberta':
#     tokenizer.additional_special_tokens = ['<s>NOTUSED', '</s>NOTUSED', args.space_token]

print('\n[INFO] Preprocess and tokenizing texts in datasets')
max_length = args.max_length if args.max_length else config.max_position_embeddings
print(f'[INFO] max_length = {max_length} \n')


[INFO] Preprocess and tokenizing texts in datasets
[INFO] max_length = 512 



In [52]:
# !pip -q install demoji
# import demoji
# demoji.download_codes()

# Load Data

In [53]:
import json
import pandas as pd

def load_jsonl(fname):
    fin = open(fname, encoding="utf-8")
    data = []
    for line in fin:
        d = json.loads(line.strip())
        data.append(d)

    return data

def save_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as fo:
        for idx, d in enumerate(data):
            fo.write(json.dumps(d, ensure_ascii=False))
            fo.write("\n")

In [54]:
ls Datasets/WisesightSentiment

neg.txt     test-misp.jsonl                  tokenized_valid.jsonl
neu.txt     tokenized_test.jsonl             train.jsonl
pos.txt     tokenized_test-misp.jsonl        valid.jsonl
q.txt       tokenized_train.jsonl
test.jsonl  tokenized_train-misp-3000.jsonl


In [55]:
traindata = load_jsonl(f"{DIR}/../../Datasets/WisesightSentiment/tokenized_train-misp-3000.jsonl")
validdata = load_jsonl(f"{DIR}/../../Datasets/WisesightSentiment/tokenized_valid.jsonl")
testdata = load_jsonl(f"{DIR}/../../Datasets/WisesightSentiment/tokenized_test-misp.jsonl")

In [56]:
# traindata[0]["segments"]

In [57]:
import itertools
def filterByMode(data, mode=None):
  output = []
  for sent in data:
    if mode is None:
      tokenized = [seg[0] for seg in sent["segments"]]
    elif mode=="corr":
      tokenized = [seg[1] for seg in sent["segments"]]
      if len(sent["misp_tokens"])==0:
        continue
    else:
      tokenized = [seg[0] for seg in sent["segments"]]
      if len(sent["misp_tokens"])==0:
        continue
    
    tokenized = list(itertools.chain(*tokenized))
  
    output.append({
        "category": sent["category"],
        "text": sent["text"],
        "tokenized": tokenized,
        "segments": sent["segments"]
    })

  return output

traindata
validdata
allTestdata = filterByMode(testdata)
corrTestdata = filterByMode(testdata, "corr")
mispTestdata = filterByMode(testdata, "misp")
len(allTestdata), len(corrTestdata), len(mispTestdata)

(2671, 880, 880)

In [58]:
for sent in testdata[1:10]:
  print(sent)
  # break

{'category': 'neu', 'text': 'ครับ #phithanbkk', 'misp_tokens': [], 'tokenized': ['ครับ', ' ', '#phithanbkk'], 'segments': [[['ครับ', ' ', '#phithanbkk'], ['ครับ', ' ', '#phithanbkk']]]}
{'category': 'neg', 'text': 'การด่าไปเหมือนได้บรรเทาความเครียดเฉยๆ แต่บีทีเอส (รถไฟฟ้า) มันสำนึกมั้ย ก็ไม่อ่ะ 😕', 'misp_tokens': [{'corr': 'ไหม', 'misp': 'มั้ย', 'int': True, 's': 67, 't': 71}], 'tokenized': ['การ', 'ด่า', 'ไป', 'เหมือน', 'ได้', 'บรรเทา', 'ความ', 'เครียด', 'เฉย', 'ๆ', ' ', 'แต่', 'บีทีเอส', ' ', '(', 'รถ', 'ไฟฟ้า', ')', ' ', 'มัน', 'สำนึก', 'มั้ย', ' ', 'ก็', 'ไม่', 'อ่ะ', ' ', 'confused', ' ', 'face'], 'segments': [[['การ', 'ด่า', 'ไป', 'เหมือน', 'ได้', 'บรรเทา', 'ความ', 'เครียด', 'เฉย', 'ๆ', ' ', 'แต่', 'บีทีเอส', ' ', '(', 'รถ', 'ไฟฟ้า', ')', ' ', 'มัน', 'สำนึก'], ['การ', 'ด่า', 'ไป', 'เหมือน', 'ได้', 'บรรเทา', 'ความ', 'เครียด', 'เฉย', 'ๆ', ' ', 'แต่', 'บีทีเอส', ' ', '(', 'รถ', 'ไฟฟ้า', ')', ' ', 'มัน', 'สำนึก']], [['มั้ย'], ['ไหม']], [['ก็', 'ไม่', 'อ่ะ', ' ', 'confused', ' ', 'fac

In [59]:
import glob
from torch.utils.data import Dataset as TorchDataset
from datasets import Dataset
# from thai2transformers.datasets import SequenceClassificationDataset

class SequenceClassificationDataset(TorchDataset):
    def __init__(
        self,
        tokenizer,
        data_dir,
        task=Task.MULTICLASS_CLS,
        max_length=128,
        ext=".csv",
        bs=10000,
        preprocessor=None,
        input_ids=[],
        misp_ids=[],
        attention_masks=[],
        labels=[],
        label_encoder=None
    ):
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.bs = bs
        self.preprocessor = preprocessor
        self.input_ids = input_ids
        self.misp_ids = misp_ids
        self.attention_masks = attention_masks
        self.labels = labels
        self.task = task
        self.label_encoder = label_encoder
        # self._build()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return {
            "input_ids": torch.tensor(self.input_ids[i], dtype=torch.long),
            "misp_ids": torch.tensor(self.misp_ids[i], dtype=torch.long),
            "attention_mask": torch.tensor(self.attention_masks[i], dtype=torch.long),
            "label": torch.tensor(self.labels[i], dtype=torch.long),
        }


Add additional tokens

In [60]:
tokenizer.add_tokens(["<rep>", "<int>", "<misp>", "<lol>"])
model.resize_token_embeddings(len(tokenizer)) 

Embedding(25009, 768)

In [61]:
MD = load_jsonl(f"{DIR}/../../train_mispelling_dection.jsonl")[0]

In [62]:
from tqdm import tqdm 
import itertools
from itertools import groupby
from collections import defaultdict

LABELS = {
    "neg": 2,
    "neu": 1,
    "pos": 0,
    "q": 1 # used to be 3
}

class CustomLabelEncoder():
    def __init__(self):
        pass

    def transform(self, labels):
        return [LABELS[l] for l in labels]
    

def remove_starting_marker(t, unk):
    if len(t) > 0:
      if t[0]=='▁':
        t = t[1:]
      elif t[0].startswith('▁'):        
        if tokenizer.convert_tokens_to_ids([t[0][1:]])[0] != unk:
          t[0] = t[0][1:]
    return t

def norm_word(word, haha=False, notoken=False):
    groups = [list(s) for _, s in groupby(word)]
    ch = []
    extraToken = ""
    for g in groups:
        if len(g)>=3:
            if g[0]=="5" and haha:
              ch.append("555")  
              extraToken = "<lol>"
            else:
              extraToken = "<rep>"
              ch.append(g[0])  
        else:
            ch += g

    word = "".join(ch)+extraToken
    if notoken:
      word = "".join(ch)
    
    return word


def preprocessing(d, preSegmented=False, mode=None):
    max_length = 400
    custom_label_encoder = CustomLabelEncoder()
    labels = get_dict_val(d, "category")

    labels = custom_label_encoder.transform(labels)

    input_ids = []
    misp_ids = []
    attention_masks = []
    unk = tokenizer.convert_tokens_to_ids(["<unk>"])[0]

    cnt = defaultdict(int)
    sents = []
    if not preSegmented:
      texts = get_dict_val(d, "tokenized")
      for tokens in tqdm(texts):
        tokens = [(t, t) for t in tokens]
        sents.append(tokens)
        
    else:
      texts = get_dict_val(d, "segments")
      for segments in tqdm(texts):
        s = [list(zip(seg[0], seg[1])) for seg in segments]
        tokens = list(itertools.chain(*s))
        sents.append(tokens)

    for tokens in sents:
      # if mode is None, ignore corr
      cnt["tokens"] += len(tokens)
      misptokens = [t[0] for t in tokens]
      corrtokens = [t[0] for t in tokens]
      
      if mode=="corr":
        misptokens = [t[1] for t in tokens]
        corrtokens = [t[1] for t in tokens]
      elif mode=="mae":
        misptokens = [t[0] for t in tokens]
        corrtokens = [t[1] for t in tokens]
      
      
      midx = tokenizer.convert_tokens_to_ids(misptokens)
      cidx = tokenizer.convert_tokens_to_ids(corrtokens)
      assert(len(midx)==len(cidx))

      newmisptokens = []
      newcorrtokens = []
      for i in range(len(midx)):
          if midx[i]==unk:
              t = norm_word(misptokens[i], True, notoken=True)
              t = tokenizer.tokenize(t)
              t = remove_starting_marker(t, unk)

              if misptokens[i]==corrtokens[i]:
                newmisptokens += t
                newcorrtokens += t
              else:
                newmisptokens += t
                tx = norm_word(corrtokens[i], True, notoken=True)
                tx = tokenizer.tokenize(tx)
                tx = remove_starting_marker(tx, unk)

                if len(tx) > 0:
                  newcorrtokens += [tx[0] for j in range(len(t))]
                else:
                  newcorrtokens += t
          else:
              t = norm_word(misptokens[i], True, notoken=True)
              tx = norm_word(corrtokens[i], True, notoken=True)

              newmisptokens.append(t)
              newcorrtokens.append(tx)
          
          norm = norm_word(misptokens[i])
          if "<rep>" in norm:
            n = norm_word(misptokens[i], True)
            if "<rep>" in n:
              newmisptokens.append("<rep>")
              newcorrtokens.append("<rep>")
              cnt["<rep>"] += 1
            elif "<lol>" in n:
              newmisptokens.append("<lol>")
              newcorrtokens.append("<lol>")
              cnt["<lol>"] += 1

          if norm in MD:
            corr, mint = MD[norm]
            if mint:
              newmisptokens.append("<int>")
              newcorrtokens.append("<int>")
              cnt["<int>"] += 1
            else:
              newmisptokens.append("<msp>")
              newcorrtokens.append("<msp>")
              cnt["<misp>"] += 1

      
            
      
      assert(len(newmisptokens)==len(newcorrtokens))
      cnt["extra"] += (len(newmisptokens) - len(misptokens))
              
      # words = newwords
      newmisptokens = ['<s>'] + newmisptokens[0:max_length-2] + ['</s>']
      newcorrtokens = ['<s>'] + newcorrtokens[0:max_length-2] + ['</s>']
    
      midx = tokenizer.convert_tokens_to_ids(newmisptokens)
      cidx = tokenizer.convert_tokens_to_ids(newcorrtokens)

      mask = [1 for i in midx]
        
      input_ids.append(midx)
      misp_ids.append(cidx)
      attention_masks.append(mask)

    #   if len(input_ids) > 10:
    #     break
    
    # labels = labels[0:10]

    for k in cnt:
      print("Number of", k, cnt[k])
    return SequenceClassificationDataset(
        tokenizer=tokenizer,
        data_dir=None,
        max_length=max_length,
        input_ids=input_ids,
        misp_ids=misp_ids,
        attention_masks=attention_masks,
        labels=labels,
        task=task
    )

raw_datasets = {
    "train": traindata,
    "validation": validdata,
    "test": allTestdata,
    "test-corr": corrTestdata,
    "test-misp": mispTestdata,
    "test-mae": mispTestdata,
    "test-all-mae": allTestdata,
}

dataset_split = {}
for split_name in raw_datasets:
    print("Tokenizing:", split_name)
    d = pd.DataFrame(raw_datasets[split_name])
    d = Dataset.from_pandas(d)
    preSegmented = split_name.startswith("test") or split_name.startswith("train")
    mode = None
    if "corr" in split_name:
      mode = "corr"
    elif "misp" in split_name:
      mode = "misp"
    elif "mae" in split_name:
      mode = "mae"

    dataset_split[split_name] = preprocessing(d, preSegmented=preSegmented, mode=mode)
    # break

Tokenizing: train


100%|██████████| 3000/3000 [00:00<00:00, 99074.93it/s]


Number of tokens 78468
Number of <rep> 669
Number of <int> 1126
Number of extra 3432
Number of <lol> 245
Number of <misp> 695
Tokenizing: validation


100%|██████████| 2404/2404 [00:00<00:00, 162932.97it/s]


Number of tokens 67236
Number of <rep> 523
Number of <int> 441
Number of extra 2819
Number of <lol> 177
Number of <misp> 404
Tokenizing: test


...

Number of tokens 22224
Number of <misp> 157
Number of extra 1664
Number of <int> 485
Number of <rep> 324
Number of <lol> 72
Tokenizing: test-mae


100%|██████████| 880/880 [00:00<00:00, 105916.77it/s]


Number of tokens 22224
Number of <misp> 157
Number of extra 1664
Number of <int> 485
Number of <rep> 324
Number of <lol> 72
Tokenizing: test-all-mae


100%|██████████| 2671/2671 [00:00<00:00, 91270.41it/s]


Number of tokens 74865
Number of <misp> 402
Number of extra 4227
Number of <int> 646
Number of <rep> 600
Number of <lol> 221


## Custom Classes for Our Experiments

In [63]:
from transformers.modeling_roberta import RobertaEmbeddings

In [64]:
from torch import nn

class CustomRobertaEmbeddings(RobertaEmbeddings):

    def __init__(self, ref, config):
        super().__init__(config)
        self.word_embeddings = ref.word_embeddings

    def forward(self, input_ids, misp_ids, token_type_ids=None, position_ids=None, inputs_embeds=None):

        input_shape = input_ids.size()
        seq_length = input_shape[1]

        inputs_embeds = self.word_embeddings(input_ids)
        misp_embeds = self.word_embeddings(misp_ids)

        embeddings = ((inputs_embeds + misp_embeds)*0.5)
        return embeddings


In [65]:
from transformers.modeling_roberta import RobertaModel

In [66]:
from transformers.modeling_camembert import CamembertForSequenceClassification
from transformers.modeling_roberta import RobertaForSequenceClassification
from torch.nn import CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutput


class CustomSequenceClassification(CamembertForSequenceClassification):
    authorized_missing_keys = [r"position_ids"]

    def __init__(self, config, refmodel=None):
        super().__init__(config)
        if refmodel is not None:
          config = refmodel.config
          # self.refmodel = refmodel
          self.num_labels = config.num_labels

          self.roberta = refmodel.roberta
          self.classifier = refmodel.classifier

          self.baseEmb = refmodel.roberta.embeddings
          self.newEmb = CustomRobertaEmbeddings(self.baseEmb, config)

    def forward(self, *args, **kwargs):
        # del kwargs["misp_ids"]
        # return self.refmodel(**kwargs)

        return_dict = self.config.use_return_dict

        inputs_embeds = self.newEmb(kwargs["input_ids"], kwargs["misp_ids"])
        
        # doc: https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel.forward
        outputs = self.roberta(
            input_ids=None,
            attention_mask=kwargs["attention_mask"],
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=inputs_embeds,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]
        logits = self.classifier(sequence_output)

        loss = None
        labels = kwargs["labels"]
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [67]:
# cusmodel = CustomSequenceClassification(model.config, model)
# trainer, training_args = init_trainer(task=task,
#                             model=cusmodel,
#                             train_dataset=dataset_split['train'],
#                             val_dataset=dataset_split['validation'] if 'validation' in DATASET_METADATA[args.dataset_name]['split_names'] else None,
#                             warmup_steps=warmup_steps,
#                             args=args,
#                             data_collator=data_collator)

# p, label_ids, result = trainer.predict(test_dataset=dataset_split['test'])

# print(f'Evaluation on test set (dataset: {args.dataset_name})')    

# for key, value in result.items():
#     print(f'{key} : {value:.4f}')

# # Evaluation on test set (dataset: wisesight_sentiment)
# # eval_loss : 1.0284
# # eval_accuracy : 0.3000
# # eval_f1_micro : 0.3000
# # eval_precision_micro : 0.3000
# # eval_recall_micro : 0.3000
# # eval_f1_macro : 0.1667
# # eval_precision_macro : 0.1667
# # eval_recall_macro : 0.1667
# # eval_nb_samples : 10.0000

In [68]:
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
from transformers import PreTrainedTokenizerBase

@dataclass
class CustomDataCollatorWithPadding:
  tokenizer: PreTrainedTokenizerBase
  padding: Union[bool, str] = True
  max_length: Optional[int] = None
  pad_to_multiple_of: Optional[int] = None
  return_tensors: str = "pt"

  def __call__(self, features):
    _tmpfeat = []
    for f in features:
      _tmpfeat.append({
          "input_ids": f["input_ids"],
          "attention_mask": f["attention_mask"],
          "label": f["label"],
      })

    batch = self.tokenizer.pad(
        _tmpfeat,
        padding=self.padding,
        max_length=self.max_length,
        pad_to_multiple_of=self.pad_to_multiple_of,
        # return_tensors=self.return_tensors,
    )

    _tmpfeat = []
    for f in features:
      _tmpfeat.append({
          "input_ids": f["misp_ids"],
          "attention_mask": f["attention_mask"],
          "label": f["label"],
      })
      
    mispbatch = self.tokenizer.pad(
        _tmpfeat,
        padding=self.padding,
        max_length=self.max_length,
        pad_to_multiple_of=self.pad_to_multiple_of,
        # return_tensors=self.return_tensors,
    )

    # print(mispbatch["input_ids"])
    # print(tokenizer.convert_ids_to_tokens(batch["input_ids"][0]))
    # print(tokenizer.convert_ids_to_tokens(mispbatch["input_ids"][0]))
    batch["misp_ids"] = mispbatch["input_ids"]
    assert(batch["misp_ids"].shape==batch["input_ids"].shape)
    # assert()
    if "label" in batch:
        batch["labels"] = batch["label"]
        del batch["label"]
    if "label_ids" in batch:
        batch["labels"] = batch["label_ids"]
        del batch["label_ids"]
    return batch

# Model Training 

In [69]:
warmup_steps = math.ceil(len(dataset_split['train']) / args.batch_size * args.warmup_ratio * args.num_train_epochs)

print(f'\n[INFO] Number of train examples = {len(raw_datasets["train"])}')
print(f'[INFO] Number of batches per epoch (training set) = {math.ceil(len(dataset_split["train"]) / args.batch_size)}')

print(f'[INFO] Warmup ratio = {args.warmup_ratio}')
print(f'[INFO] Warmup steps = {warmup_steps}')
print(f'[INFO] Learning rate: {args.learning_rate}')
print(f'[INFO] Logging steps: {args.logging_steps}')
print(f'[INFO] FP16 training: {args.fp16}\n')

# if 'validation' in DATASET_METADATA[args.dataset_name]['split_names']:
print(f'[INFO] Number of validation examples = {len(raw_datasets["validation"])}')
print(f'[INFO] Number of batches per epoch (validation set) = {math.ceil(len(dataset_split["validation"]))}')

data_collator = CustomDataCollatorWithPadding(tokenizer,
                                        padding=True,
                                        pad_to_multiple_of=8 if args.fp16 else None)

cusmodel = CustomSequenceClassification(model.config, model)
trainer, training_args = init_trainer(task=task,
                            model=cusmodel,
                            train_dataset=dataset_split['train'],
                            val_dataset=dataset_split['validation'] if 'validation' in DATASET_METADATA[args.dataset_name]['split_names'] else None,
                            warmup_steps=warmup_steps,
                            args=args,
                            data_collator=data_collator)

print('[INFO] TrainingArguments:')
print(training_args)
print('\n')


[INFO] Number of train examples = 3000
[INFO] Number of batches per epoch (training set) = 375
[INFO] Warmup ratio = 0.1
[INFO] Warmup steps = 375
[INFO] Learning rate: 1e-05
[INFO] Logging steps: 10
[INFO] FP16 training: False

[INFO] Number of validation examples = 2404
[INFO] Number of batches per epoch (validation set) = 2404
[INFO] TrainingArguments:
TrainingArguments(output_dir='Models/WangchanBERTa-exp4/Outputs/', overwrite_output_dir=True, do_train=False, do_eval=None, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10, max_steps=-1, warmup_steps=375, logging_dir='Models/WangchanBERTa

In [70]:
print('\nBegin model finetuning.')
trainer.train()
print('Done.\n')


Begin model finetuning.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,Precision Micro,Recall Micro,F1 Macro,Precision Macro,Recall Macro,Nb Samples
1,0.979016,0.935150,0.555740,0.555740,0.555740,0.555740,0.507539,0.504584,0.530383,2404
2,0.916223,0.846718,0.576123,0.576123,0.576123,0.576123,0.577391,0.607450,0.645955,2404
3,0.734912,0.850426,0.611065,0.611065,0.611065,0.611065,0.591297,0.596512,0.642996,2404
4,0.595203,0.817019,0.634359,0.634359,0.634359,0.634359,0.628073,0.636871,0.683007,2404
5,0.646594,0.784293,0.664725,0.664725,0.664725,0.664725,0.644470,0.636201,0.676973,2404
6,0.528430,0.781751,0.668885,0.668885,0.668885,0.668885,0.649781,0.644296,0.680282,2404
7,0.476794,0.837725,0.665141,0.665141,0.665141,0.665141,0.649830,0.644305,0.689222,2404
8,0.549316,0.890759,0.652246,0.652246,0.652246,0.652246,0.639569,0.638745,0.680537,2404
9,0.428149,0.931990,0.642679,0.642679,0.642679,0.642679,0.633422,0.636639,0.679965,2404
10,0.401685,0.921691,0.652662,0.652662,0.652662,0.652662,0.641077,0.641050,0.683075,2404


Some weights of the model checkpoint at Models/WangchanBERTa-exp4/Outputs/checkpoint-2250 were not used when initializing CustomSequenceClassification: ['baseEmb.position_ids', 'baseEmb.word_embeddings.weight', 'baseEmb.position_embeddings.weight', 'baseEmb.token_type_embeddings.weight', 'baseEmb.LayerNorm.weight', 'baseEmb.LayerNorm.bias', 'newEmb.position_ids', 'newEmb.word_embeddings.weight', 'newEmb.position_embeddings.weight', 'newEmb.token_type_embeddings.weight', 'newEmb.LayerNorm.weight', 'newEmb.LayerNorm.bias']
- This IS expected if you are initializing CustomSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClass

Done.



In [71]:
trainer.save_model(f"{DIR}/fine-tune-Exp4")

# Evaluation

In [72]:
import warnings
warnings.filterwarnings("ignore")

In [73]:
# (model.classifier.dense.weight.shape-trainer.model.classifier.dense.weight.shape).sum()

In [74]:
# assert(torch.equal(model.classifier.dense.weight, trainer.model.classifier.dense.weight))
trainer.model = CustomSequenceClassification(model.config, model)

In [75]:
trainer.model.eval();

In [76]:

for split_name in dataset_split:
  if split_name.startswith("train"):
    continue

  p, label_ids, result = trainer.predict(test_dataset=dataset_split[split_name])
  print(f'Evaluation on {split_name}:')    

  for key, value in result.items():
      print(f'{key} : {value:.4f}')
  
  print("*"*40)
  print()

Evaluation on validation:
eval_loss : 0.9217
eval_accuracy : 0.6527
eval_f1_micro : 0.6527
eval_precision_micro : 0.6527
eval_recall_micro : 0.6527
eval_f1_macro : 0.6411
eval_precision_macro : 0.6410
eval_recall_macro : 0.6831
eval_nb_samples : 2404.0000
****************************************

Evaluation on test:
eval_loss : 0.8919
eval_accuracy : 0.6593
eval_f1_micro : 0.6593
eval_precision_micro : 0.6593
eval_recall_micro : 0.6593
eval_f1_macro : 0.6453
eval_precision_macro : 0.6419
eval_recall_macro : 0.6875
eval_nb_samples : 2671.0000
****************************************

Evaluation on test-corr:
eval_loss : 0.9176
eval_accuracy : 0.6670
eval_f1_micro : 0.6670
eval_precision_micro : 0.6670
eval_recall_micro : 0.6670
eval_f1_macro : 0.6650
eval_precision_macro : 0.6684
eval_recall_macro : 0.7027
eval_nb_samples : 880.0000
****************************************

Evaluation on test-misp:
eval_loss : 0.9588
eval_accuracy : 0.6432
eval_f1_micro : 0.6432
eval_precision_micro : 0